In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import pytube
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi 

### Download Video using PyTube

In [2]:
VIDEO_URL = 'https://youtu.be/2V7yPrxJ8Ck?si=h2nmLsscinypIesK'

In [3]:
def downloadYTvideo(url, save_dir="../data"):
    yt = YouTube(url)
    video_title = yt.title
    video_id = pytube.extract.video_id(VIDEO_URL)
    # Issue: https://github.com/pytube/pytube/issues/1626
    yt.streams.first()

    video_description = yt.description
    video = yt.streams.filter(progressive=True).get_highest_resolution()
    try:
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        print(f"Downloading video titled: {video_title}...")
        video.download(save_dir)
        print("Videp Downloaded.")

        print('Downloading transcript')
        transcript_info = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        df = pd.DataFrame.from_dict(transcript_info) 
        df.to_csv(os.path.join(save_dir, 'transcript.csv'), index = False)
        print('Transcript saved.')
        
    except:
        print("Failed to download video")

In [4]:
downloadYTvideo(VIDEO_URL)

Videp Downloaded.
Transcript saved.


### Load Dataset

In [6]:
df = pd.read_csv('../data/transcript.csv')

In [7]:
df.head(5)

,text,start,duration
0,just improve your problem solving skills,1.599,3.521
1,and you'll be able to pass all your,3.719,3.401
2,coding interviews just solve some,5.120,3.920
3,linkless questions over here and some,7.120,3.639
4,tree questions over there and then,9.040,3.679


In [12]:
video_text_corpus = df.text.str.cat(sep = ' ')

In [13]:
print('Length of video: ', len(video_text_corpus))

Length of video:  5032


### Phi3 For HyDe